In [1]:
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler #labeling, scaling
from sklearn.model_selection import train_test_split #data split
from sklearn.metrics import accuracy_score, confusion_matrix #checking
from sklearn.linear_model import LogisticRegression #model

### Data Import - Iris

In [2]:
iris=sns.load_dataset('iris')
X=iris.drop('species',axis=1)
y=iris['species']

#labeling
classle=LabelEncoder()
y=classle.fit_transform(y.values)

### Data Split

In [3]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)

### Data Scaling

In [4]:
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

### LogisticRegression
* penalty : 규제화 종류 선택. 'l1','l2','elasticnet' and 'none'
* C : 1/λ. C가 작을수록 강한 규제. 양의 실수여야 함.
* solver : 최적화 사용될 알고리즘 선택. ‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’ and ‘saga’
    * 소표본의 경우 'liblinear'가 좋지만, 대표본의 경우 'sag' 혹은 'saga'가 더 빠르다.
    * 'multinomial'의 경우 'liblinear'를 사용할 수 없다.
    * penalty에 따라 사용가능한 solver가 다르므로 확인 필요.
* multi_class : ‘ovr’, ‘multinomial’ and ‘auto’

In [5]:
# 규제화에 따른 정확도 비교.
for pen in ['l1','l2']:
    logit=LogisticRegression(C=1e2,random_state=10,penalty=pen,solver='liblinear',multi_class='ovr')
    logit.fit(X_train_std,y_train)
    y_test_pred=logit.predict(X_test_std)
    print('{} : {}'.format(pen,accuracy_score(y_test,y_test_pred)))

l1 : 0.9555555555555556
l2 : 0.9777777777777777


In [6]:
# λ에 따른 정확도 비교.
# λ가 너무 크거나(작은 C값, 강한 규제) λ가 너무 작으면(큰 C값, 약한 규제) 정확도가 떨어짐.
for c in [1e4,1e2,1,1e-2]:
    logit=LogisticRegression(C=c,random_state=10,solver='lbfgs',multi_class='ovr')
    logit.fit(X_train_std,y_train)
    y_test_pred=logit.predict(X_test_std)
    print('c={} : {}'.format(c,accuracy_score(y_test,y_test_pred)))

c=10000.0 : 0.9333333333333333
c=100.0 : 0.9777777777777777
c=1 : 0.9555555555555556
c=0.01 : 0.8888888888888888


In [7]:
# binary 혹은 multi에 따른 정확도 비교.
for mcl in ['ovr','multinomial','auto']:
    logit=LogisticRegression(C=1e2,random_state=10,solver='lbfgs',multi_class=mcl)
    logit.fit(X_train_std,y_train)
    y_test_pred=logit.predict(X_test_std)
    print('{} : {}'.format(mcl,accuracy_score(y_test,y_test_pred)))

ovr : 0.9777777777777777
multinomial : 0.9555555555555556
auto : 0.9555555555555556
